In [4]:
%matplotlib
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
print(mpl.get_backend())
import matplotlib.pyplot as plt
print(mpl.get_backend())
from utils.redraw_obstacles import redraw_obstacles, set_non_blocking
from utils.create_draw_params import create_draw_params
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np

from commonroad.prediction.prediction import Prediction, Occupancy
def occupancy_at_time(self, time_step: int):
    """
    Returns the predicted occupancy of the obstacle at a specific time step.

    :param time_step: discrete time step
    :return: predicted occupancy of the obstacle at time step
    """
    occupancy = None
    if time_step == self.initial_state.time_step:
        shape = self.obstacle_shape.rotate_translate_local(
            self.initial_state.position, self.initial_state.orientation)
        occupancy = Occupancy(time_step, shape)
    elif time_step > self.initial_state.time_step and time_step < (self.initial_state.time_step + len(self._prediction.trajectory.state_list)) and   self._prediction is not None:
        occupancy = self._prediction.occupancy_at_time_step(time_step)
    return occupancy

from commonroad.scenario.obstacle import StaticObstacle, DynamicObstacle, Obstacle
DynamicObstacle.occupancy_at_time = occupancy_at_time

print(mpl.get_backend())

Using matplotlib backend: Qt5Agg
TkAgg
TkAgg
TkAgg


In [5]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/minicity.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()

In [3]:
import copy
scenario_bak =  copy.deepcopy(scenario)

In [17]:
scenario = copy.deepcopy(scenario_bak)

In [7]:
def create_draw_params() -> dict:
    basic_shape_parameters_static = {'opacity': 1.0,
                                   'facecolor': '#000000',
                                   'edgecolor': '#000000',
                                   'linewidth': 0.5,
                                   'zorder': 20}

    basic_shape_parameters_dynamic = {'opacity': 1.0,
                                     'facecolor': '#000000',
                                     'edgecolor': '#000000',
                                     'linewidth':0.5,
                                     'zorder': 20}

    draw_params = {'scenario': {
                        'dynamic_obstacle': {
                            'draw_shape': True,
                            'draw_icon': False,
                            'draw_bounding_box': True,
                            'show_label': False,
                            'trajectory_steps': 0,
                            'zorder': 20,
                            'occupancy': {
                                'draw_occupancies': -1,  # -1= never, 0= if prediction of vehicle is set-based, 1=always
                                'shape': {
                                    'polygon': {
                                    'opacity': 0.2,
                                       'facecolor': '#1d7eea',
                                       'edgecolor': '#0066cc',
                                       'linewidth': 0.5,
                                       'zorder': 18,
                                    },
                                    'rectangle': {
                                       'opacity': 0.2,
                                       'facecolor': '#1d7eea',
                                       'edgecolor': '#0066cc',
                                       'linewidth': 0.5,
                                       'zorder': 18,
                                    },
                                    'circle': {
                                       'opacity': 0.2,
                                       'facecolor': '#1d7eea',
                                       'edgecolor': '#0066cc',
                                       'linewidth': 0.5,
                                       'zorder': 18,
                                    }
                                },
                            },
                            'shape': {
                                'polygon': basic_shape_parameters_dynamic,
                                'rectangle': basic_shape_parameters_dynamic,
                                'circle': basic_shape_parameters_dynamic
                            },
                             'trajectory': {'facecolor': '#000000'}
                        },
                        'static_obstacle': {
                           'shape': {
                               'polygon': basic_shape_parameters_static,
                               'rectangle':basic_shape_parameters_static,
                               'circle': basic_shape_parameters_static,
                           }
                        },
                        'lanelet_network': {
                            'lanelet': {'left_bound_color': '#FFFFFF',
                                       'right_bound_color': '#FFFFFF',
                                       'center_bound_color': '#FFFFFF',
                                       'draw_left_bound': True,
                                       'draw_right_bound': True,
                                       'draw_center_bound': False,
                                       'draw_border_vertices': False,
                                       'draw_start_and_direction': False,
                                       'show_label': False,
                                       'draw_linewidth': 5,
                                       'fill_lanelet': True,
                                       'facecolor': '#FFFFFF'}},
                   }
    }
    # ensure that parameters are also available on higher levels
    draw_params.update(draw_params['scenario'])
    draw_params['shape'] = basic_shape_parameters_static
    draw_params['shape'].update(draw_params['scenario']['static_obstacle']['shape'])
    draw_params['occupancy'] = draw_params['scenario']['dynamic_obstacle']['occupancy']
    draw_params['static_obstacle'] = draw_params['scenario']['static_obstacle']
    draw_params['dynamic_obstacle'] = draw_params['scenario']['dynamic_obstacle']
    draw_params['trajectory'] = draw_params['scenario']['dynamic_obstacle']['trajectory']
    draw_params['lanelet_network'] = draw_params['scenario']['lanelet_network']
    draw_params['lanelet'] = draw_params['scenario']['lanelet_network']['lanelet']
    draw_params['scenario']['lanelet'] = draw_params['scenario']['lanelet_network']['lanelet']
#     basic_shape_parameters_static = {'opacity': 1.0,
#                                    'facecolor': '#d95558',
#                                    'edgecolor': '#831d20',
#                                    'linewidth': 0.5,
#                                    'zorder': 20}

#     basic_shape_parameters_dynamic = {'opacity': 1.0,
#                                      'facecolor': '#1d7eea',
#                                      'edgecolor': '#0066cc',
#                                      'linewidth':0.5,
#                                      'zorder': 20}

#     draw_params = {'scenario': {
#                         'dynamic_obstacle': {
#                             'draw_shape': True,
#                             'draw_icon': False,
#                             'draw_bounding_box': True,
#                             'show_label': False,
#                             'trajectory_steps': 40,
#                             'zorder': 20,
#                             'occupancy': {
#                                 'draw_occupancies': 0,  # -1= never, 0= if prediction of vehicle is set-based, 1=always
#                                 'shape': {
#                                     'polygon': {
#                                     'opacity': 0.2,
#                                        'facecolor': '#1d7eea',
#                                        'edgecolor': '#0066cc',
#                                        'linewidth': 0.5,
#                                        'zorder': 18,
#                                     },
#                                     'rectangle': {
#                                        'opacity': 0.2,
#                                        'facecolor': '#1d7eea',
#                                        'edgecolor': '#0066cc',
#                                        'linewidth': 0.5,
#                                        'zorder': 18,
#                                     },
#                                     'circle': {
#                                        'opacity': 0.2,
#                                        'facecolor': '#1d7eea',
#                                        'edgecolor': '#0066cc',
#                                        'linewidth': 0.5,
#                                        'zorder': 18,
#                                     }
#                                 },
#                             },
#                             'shape': {
#                                 'polygon': basic_shape_parameters_dynamic,
#                                 'rectangle': basic_shape_parameters_dynamic,
#                                 'circle': basic_shape_parameters_dynamic
#                             },
#                              'trajectory': {'facecolor': '#000000'}
#                         },
#                         'static_obstacle': {
#                            'shape': {
#                                'polygon': basic_shape_parameters_static,
#                                'rectangle':basic_shape_parameters_static,
#                                'circle': basic_shape_parameters_static,
#                            }
#                         },
#                         'lanelet_network': {
#                             'lanelet': {'left_bound_color': '#555555',
#                                        'right_bound_color': '#555555',
#                                        'center_bound_color': '#dddddd',
#                                        'draw_left_bound': True,
#                                        'draw_right_bound': True,
#                                        'draw_center_bound': True,
#                                        'draw_border_vertices': False,
#                                        'draw_start_and_direction': True,
#                                        'show_label': False,
#                                        'draw_linewidth': 0.5,
#                                        'fill_lanelet': True,
#                                        'facecolor': '#c7c7c7'}},
#                    }
#     }
#     # ensure that parameters are also available on higher levels
#     draw_params.update(draw_params['scenario'])
#     draw_params['shape'] = basic_shape_parameters_static
#     draw_params['shape'].update(draw_params['scenario']['static_obstacle']['shape'])
#     draw_params['occupancy'] = draw_params['scenario']['dynamic_obstacle']['occupancy']
#     draw_params['static_obstacle'] = draw_params['scenario']['static_obstacle']
#     draw_params['dynamic_obstacle'] = draw_params['scenario']['dynamic_obstacle']
#     draw_params['trajectory'] = draw_params['scenario']['dynamic_obstacle']['trajectory']
#     draw_params['lanelet_network'] = draw_params['scenario']['lanelet_network']
#     draw_params['lanelet'] = draw_params['scenario']['lanelet_network']['lanelet']
#     draw_params['scenario']['lanelet'] = draw_params['scenario']['lanelet_network']['lanelet']

    return draw_params

In [5]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    handles = {}
    draw_params = create_draw_params()
    draw_params['time_begin'] = 0
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()

In [6]:
show_lane(scenario)

In [2]:
def plt_window(sce, dur, draw_params, size, pos):
    set_non_blocking()  # ensures interactive plotting is activated
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    ax.set_facecolor("black")
#     ax.set_cmap("binary")
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_params['time_begin'] = dur[0]
    draw_object(sce, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
    # plt.gray()
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    data = np.zeros(( (duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(dur[0], dur[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(sce, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8)
        buffer = buffer.reshape(fig.canvas.get_width_height()[::-1] + (4,))
#         print(buffer.shape)
        buffer = buffer[:, : , 0].astype(bool)
        data[(i-dur[0]), :, : ] = buffer
        print('fps:', (i - dur[0])/(time.time()-t1))
    return data

In [3]:
size = [200, 200]
pos = [100, 95]
duration = [0, 100]
inch_in_cm = 20
draw_params = create_draw_params()

origin_data = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

NameError: name 'create_draw_params' is not defined

In [8]:
def plt_data(observe):
    fig = plt.figure()
    plt.gray()
    if len(observe.shape) <=2:
        plt.imshow(observe)
    elif len(observe.shape) > 3:
        for r in observe:
            myobj = plt.imshow(r[0, :, :])
            for rr in r:
                myobj.set_data(rr)
                plt.draw()
                plt.pause(0.01)
    else:
        myobj = plt.imshow(observe[0, :, :])
        for row in observe:
            myobj.set_data(row)
            plt.draw()
            plt.pause(0.01)

In [8]:
def theTrueData(leng, start_egoid, dt):
    #  初始化fig
    set_non_blocking()  # ensures interactive plotting is activated
    size = [120, 120]
    pos = [0, 0]
    inch_in_cm = 15
    fig = plt.figure(figsize=(size[0] / inch_in_cm, size[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')

    #     draw scenario
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    draw_params = create_draw_params()
    draw_object(scenario.lanelet_network, handles=handles, draw_params=draw_params)
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    #   初始化data
    start_goal = np.zeros((leng, 8), dtype=np.float64)
    observed = np.zeros(( (leng, dt,) + fig.canvas.get_width_height()[::-1]), dtype=np.bool)
    data = np.zeros((leng,  dt, 6), dtype=np.float64)
    traj = []

    #   从哪辆车开始呢
    ego_id = start_egoid
    index = 0
    while True:
        ego = None
        while(scenario.obstacle_by_id(ego_id) == None):
            ego_id = ego_id + 1
        
        ego = scenario.obstacle_by_id(ego_id)
        print("ego_id: ", ego_id)
    #   获得那辆车的traj，initial_time, terminated_time
        traj = ego.prediction.trajectory
        initial_time = traj.initial_time_step
        terminated_time = traj.state_list[-1].time_step
#         ego_scenario.add_objects(ego)
        scenario.remove_obstacle(ego)
        
        print("initial time", initial_time)
        print("terminated_time", terminated_time)
    #  开始循环
        start_time = initial_time
        end_time = start_time + dt


        while end_time < terminated_time - 2* dt:
            t =int( (start_time - initial_time)/2)
            pos = traj.state_list[t].position
#             print("start_time", start_time)
#             print("end_time", end_time)
    #   实际traj数据
            for i in range(0, dt):
                data[index, i, 0:4] = [traj.state_list[t+i].position[0], traj.state_list[t+i].position[1], traj.state_list[t+i].orientation, traj.state_list[t+i].velocity]
                data[index, i, 5] = traj.state_list[t+i].time_step
#                 print(traj.state_list[t+i].time_step)
    #   起点，终点
            start_goal[index, 0:4] = data[index, int(dt/2), 0:4]
            start_goal[index, 4:8] = data[index, -1, 0:4]
            
#             plt.plot(data[index, :, 0], data[index, :, 1])
#             plt.scatter(start_goal[index, 0], start_goal[index, 1])
#             plt.scatter(start_goal[index, 4], start_goal[index, 5])
            
            plot_limits = [start_goal[index, 0] - size[0]/2, start_goal[index, 0] + size[0]/2, start_goal[index, 1]+-size[1]/2,start_goal[index, 1]+ size[1]/2]
            t1 = time.time()
            # loop where obstacle positions are updated
            for i in range(0, dt):
                # ...
                # change positions of obstacles
                # ...
                draw_params['time_begin'] = int(data[index, i, 5] )
                redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
#                 尝试查看ego轨迹
#                 plt.plot(data[index, :, 0], data[index, :, 1])
                buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
#                 print("cavas pix", fig.canvas.get_width_height())
                buffer = buffer[:, : , 0].astype(bool)
    #   observed data
                observed[index, i, :, : ] = buffer
#                 print('fps:', i/(time.time()-t1), " time:", i)
            start_time = start_time + dt
            end_time = start_time + dt
            index = index + 1
            if(index == leng): break
        scenario.add_objects(ego)
        if(index == leng): break
    plt.close(fig)
    return start_goal, observed, data, ego_id

In [ ]:
import torch

leng = 100
dt = 50
#  310 408
start_ego = 309
last_ego = start_ego
# scenario = copy.deepcopy(scenario_bak)
i_data = 0 
for i_data in range(0, 45):
    start_goal_i, observed_i, data_i, last_ego =  theTrueData(leng=leng, start_egoid=last_ego + 1,  dt=dt)
    PATH = '/home/rong/disk/mydata/data_1_' + str(i_data);
    print("save ", i_data)
    torch.save({
                'index': i_data,
                'start_goal': start_goal_i,
                'observation': observed_i,
                'data': data_i,
                'egoid': last_ego,
                }, PATH)

ego_id:  310
initial time 1
terminated_time 5001
ego_id:  310
initial time 1
terminated_time 5001
save  0
ego_id:  311
initial time 1
terminated_time 5001
ego_id:  311
initial time 1
terminated_time 5001
save  1
ego_id:  312
initial time 1
terminated_time 5001
ego_id:  312
initial time 1
terminated_time 5001
save  2
ego_id:  313
initial time 1
terminated_time 5001
ego_id:  313
initial time 1
terminated_time 5001
save  3
ego_id:  314
initial time 1
terminated_time 5001
ego_id:  314
initial time 1
terminated_time 5001
save  4
ego_id:  315
initial time 1
terminated_time 5001
ego_id:  315
initial time 1
terminated_time 5001
save  5
ego_id:  316
initial time 1
terminated_time 5001
ego_id:  316
initial time 1
terminated_time 5001
save  6
ego_id:  317
initial time 1
terminated_time 5001
ego_id:  317
initial time 1
terminated_time 5001
save  7
ego_id:  318
initial time 1
terminated_time 5001
ego_id:  318
initial time 1
terminated_time 5001
save  8
ego_id:  319
initial time 1
terminated_time 50

In [ ]:
for i_data in range(39, 1000):
    start_goal_i, observed_i, data_i, last_ego =  theTrueData(leng, last_ego, dt)
    PATH = '/home/rong/disk/mydata/data_' + str(i_data);
    torch.save({
                'index': i_data,
                'start_goal': start_goal_i,
                'observation': observed_i,
                'data': data_i
                }, PATH)

In [9]:
import torch
PATH = 'data/data_' + str(i_data);
torch.save({
            'index': i_data,
            'start_goal': start_goal_i,
            'observation': observed_i,
            'data': data_i
            }, PATH)

In [81]:
plt_data(observed_i)

In [26]:
load_data = torch.load(PATH)
load_start_goal = load_data['start_goal']
load_observed = load_data["observation"]
load_data = load_data["data"]

In [80]:
plt_data(load_observed)

NameError: name 'load_observed' is not defined

In [86]:
a = Range(4)

NameError: name 'Range' is not defined

In [89]:
for x in range(0, 3):
    print(x)

0
1
2
